In [10]:
import pandas as pd

df = pd.read_csv("../../complaints-2020-01-22_08_24.csv", encoding="utf-8")

df_selected = df.loc[:, ('Product', 'Consumer complaint narrative', 'Issue', 'Company')]        

---

**Functions to clean Companies**

In [11]:
companies = df_selected.groupby("Company")["Company"].count().sort_values(ascending=False)
companies = companies[0:25]

for index, key in df_selected.iterrows():
    if df_selected['Company'][index] not in companies:
        df_selected['Company'][index] = 'None'

__Functions to clean the Consumer complaint narrative__

In [12]:
import re
import string

def clean_url(complaint):
    # to do: more regex url garbage matching
    complaint = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    complaint = re.sub('https? ?: ?// ?(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    return complaint

# Remove punctuation from complaint
def clean_punctuation(complaint):
    complaint = re.sub('[%s]' % re.escape(string.punctuation), '', complaint)
    return complaint

# Remove non-sensical characters from complaint
def clean_nonsense(complaint):
    complaint = re.sub('[''""...]', '', complaint)
    complaint = re.sub('\n', '', complaint)
    return complaint

# Remove censored words from complaint
def clean_censored(complaint):
    complaint = re.sub('[XXXX]', '', complaint)
    return complaint

# Turn everything into lowercase
def clean_lowercase(complaint):
    complaint = complaint.lower()
    return complaint

# Remove numbers from complaint
def clean_numbers(complaint):
    complaint = re.sub('\w*\d\w', '', complaint)
    return complaint

__Application of narrative cleaning__

In [13]:
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_url)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_punctuation)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_nonsense)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_censored)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_lowercase)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_numbers)

**Apply CountVectorizer**

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

**Apply TF-IDF**

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

# Normalise with tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Shape', X_train_tfidf.shape)
print(X_train_tfidf)

Shape (485701, 77541)
  (0, 77176)	0.057603117293145815
  (0, 76652)	0.16256155704199224
  (0, 75309)	0.0704998089772363
  (0, 75075)	0.06367724939199933
  (0, 74968)	0.09219266901371895
  (0, 73727)	0.12517299128438755
  (0, 71967)	0.17949182587880833
  (0, 69841)	0.11897824400517687
  (0, 69713)	0.05790316684384142
  (0, 69322)	0.11526692039409442
  (0, 69116)	0.0919644454692771
  (0, 67077)	0.0945540081983084
  (0, 66813)	0.13729499089431052
  (0, 65093)	0.10163692275565259
  (0, 63443)	0.12076334352602229
  (0, 62631)	0.062129528417944724
  (0, 62615)	0.12209465027976771
  (0, 62146)	0.11073198116064686
  (0, 61559)	0.07623732630893473
  (0, 59674)	0.09011779388563108
  (0, 59664)	0.09770742191644613
  (0, 58514)	0.05250540217562024
  (0, 58403)	0.04014663511243601
  (0, 57662)	0.14368581222288473
  (0, 56871)	0.14107643748845286
  :	:
  (485700, 72656)	0.20663930774149367
  (485700, 71225)	0.26283911340165067
  (485700, 70092)	0.0918765457662843
  (485700, 61605)	0.194740851743595

**Dimensionality Reduction**

In [16]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_tfidf)
print(principal_components.shape)

(485701, 5)


__One hot encoding of extra columns__

In [17]:
principal_components_df = pd.DataFrame(principal_components)

df_selected['Issue'] = pd.Categorical(df_selected['Issue'])
df_dummies1 = pd.get_dummies(df_selected['Issue'])

df_selected['Company'] = pd.Categorical(df_selected['Company'])
df_dummies2 = pd.get_dummies(df_selected['Company'])

df_concat = pd.concat([principal_components_df, df_dummies1, df_dummies2], axis = 1)
print(df_concat)

               0         1         2         3         4  \
0       0.187932  0.021535  0.007940 -0.000300 -0.030178   
1       0.123744  0.042944 -0.122385  0.075660 -0.014762   
2       0.177520  0.029051  0.167645 -0.003684 -0.022923   
3       0.196347  0.083276  0.261556  0.083273  0.073679   
4       0.277747  0.005680  0.134211  0.053774 -0.106588   
5       0.194632  0.027059  0.182215  0.018858 -0.039746   
6       0.155707  0.147216 -0.036642 -0.067013 -0.023963   
7       0.187515  0.221212 -0.052410 -0.070091  0.033070   
8       0.070227  0.066830 -0.025544 -0.018133 -0.026158   
9       0.194027 -0.193693  0.004704 -0.077737  0.034068   
10      0.186640  0.129039  0.131353  0.035419  0.066256   
11      0.187084 -0.099897  0.005639 -0.033637  0.005306   
12      0.206061 -0.112562 -0.076143 -0.005844  0.008569   
13      0.245871  0.154304  0.049646 -0.027587  0.026431   
14      0.200794  0.020004 -0.103318  0.045371  0.170623   
15      0.250799  0.159743 -0.170671  0.

**Split into train & test**

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(364275, 192)
(121426, 192)
(364275,)
(121426,)


**Run algorithm**

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

test_predictions = clf.predict(X_test)

**Report**

In [20]:
print(classification_report(test_predictions, Y_test))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3628
                                                 Checking or savings account       0.99      1.00      1.00      4703
                                                               Consumer Loan       0.85      0.79      0.82      2595
                                                                 Credit card       0.99      0.97      0.98      4794
                                                 Credit card or prepaid card       0.95      0.99      0.97      7554
                                                            Credit reporting       1.00      1.00      1.00      7919
Credit reporting, credit repair services, or other personal consumer reports       1.00      0.97      0.98     36406
                                                       